In [1]:
import sqlalchemy
import pandas as pd
import datetime
import numpy as np
BL_engine=sqlalchemy.create_engine(
            "mysql+pymysql://%s:%s@localhost/%s" % ('jian', 'JubaPlus-2017', 'BigLots'))

In [2]:
pd.read_sql("desc Pred_POS_Department;",con=BL_engine)

/usr/local/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


,Field,Type,Null,Key,Default,Extra
0,location_id,int(11),YES,MUL,None,
1,transaction_dt,date,YES,MUL,None,
2,transaction_id,varchar(16),YES,,None,
3,customer_id_hashed,char(64),YES,MUL,None,
4,department_id,varchar(16),YES,MUL,None,
5,sales,"decimal(10,2)",YES,,None,
6,units,int(11),YES,,None,
7,trans_order_since_18Q1,int(11),YES,MUL,None,


In [3]:
'''
df_wrong_ids=pd.read_sql("select distinct customer_id_hashed as customer_id_hashed from Pred_POS_Department where transaction_dt between '2020-06-07' and '2020-06-13'",con=BL_engine)
df_wrong_ids=df_wrong_ids[pd.notnull(df_wrong_ids['customer_id_hashed'])]
print(df_wrong_ids.shape)
df_wrong_ids.head(2)

df_wrong_ids.to_sql(name="df_wrong_ids",con=BL_engine,dtype={"customer_id_hashed":sqlalchemy.types.VARCHAR(length=64)},if_exists="replace")
'''
# 1.4 mm

'\ndf_wrong_ids=pd.read_sql("select distinct customer_id_hashed as customer_id_hashed from Pred_POS_Department where transaction_dt between \'2020-06-07\' and \'2020-06-13\'",con=BL_engine)\ndf_wrong_ids=df_wrong_ids[pd.notnull(df_wrong_ids[\'customer_id_hashed\'])]\nprint(df_wrong_ids.shape)\ndf_wrong_ids.head(2)\n\ndf_wrong_ids.to_sql(name="df_wrong_ids",con=BL_engine,dtype={"customer_id_hashed":sqlalchemy.types.VARCHAR(length=64)},if_exists="replace")\n'

In [4]:
df_wrong_ids=pd.read_sql("select customer_id_hashed from df_wrong_ids;",con=BL_engine)
list_wrong_ids=df_wrong_ids['customer_id_hashed'].values.tolist()

In [5]:
df_pre_max_order=pd.read_sql("select customer_id_hashed, max(trans_order_since_18Q1) as pre_order \
from Pred_POS_Department \
where transaction_dt <= '2020-06-06' \
group by customer_id_hashed;",con=BL_engine)

len(list_wrong_ids)

1404642

In [7]:
df_pre_max_order.shape

(27144206, 2)

In [9]:
df_pre_max_order=pd.merge(df_wrong_ids,df_pre_max_order,on="customer_id_hashed",how="left")
df_pre_max_order['pre_order']=df_pre_max_order['pre_order'].fillna(0)
print(df_pre_max_order.shape)
df_pre_max_order.head(2)

(1404642, 2)


,customer_id_hashed,pre_order
0,0000141dbeb91820de94321c672f0a4cf3b763e31739ce...,9.0
1,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,23.0


In [8]:
df_pre_max_order.to_csv("./df_pre_max_order.csv",index=False)

In [10]:
chunck_size=10**4
chunck_count=int(np.ceil(len(list_wrong_ids)/chunck_size))
df_wrong_trans=pd.DataFrame()
for i in range(chunck_count):
    chunck_list=list_wrong_ids[i*chunck_size:(i+1)*chunck_size]
    str_list_wrong_ids=str(chunck_list).replace("[","(").replace("]",")")
    query_to_delete_in="select * from Pred_POS_Department where transaction_dt >= '2020-06-07' and customer_id_hashed in %s"%str_list_wrong_ids
    df=pd.read_sql(query_to_delete_in,con=BL_engine)
    df_wrong_trans=df_wrong_trans.append(df)
    if i%10==1:
        print(i,datetime.datetime.now())
df_wrong_trans.shape

1 2020-08-11 22:17:44.555195
11 2020-08-11 22:21:55.999707
21 2020-08-11 22:28:03.141644
31 2020-08-11 22:34:15.477683
41 2020-08-11 22:40:39.956927
51 2020-08-11 22:46:51.434586
61 2020-08-11 22:53:09.967408
71 2020-08-11 22:59:30.968156
81 2020-08-11 23:05:51.076620
91 2020-08-11 23:12:19.000458
101 2020-08-11 23:18:45.437492
111 2020-08-11 23:25:00.710028
121 2020-08-11 23:31:17.758165
131 2020-08-11 23:37:39.585977


(11972429, 8)

In [11]:
df_wrong_trans.head(4)

,location_id,transaction_dt,transaction_id,customer_id_hashed,department_id,sales,units,trans_order_since_18Q1
0,5383,2020-06-08,9296,0000141dbeb91820de94321c672f0a4cf3b763e31739ce...,353,5.0,2,11
1,1532,2020-06-12,6125,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,110,3.0,1,25
2,1532,2020-06-12,6125,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,111,1.9,1,25
3,1532,2020-06-12,6125,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,140,3.0,3,25


In [20]:
df_wrong_trans['customer_id_hashed'].nunique()

1404642

In [12]:
df_wrong_id_pre_order=pd.merge(df_wrong_ids,df_pre_max_order,on="customer_id_hashed",how="left")
df_wrong_id_pre_order['pre_order']=df_wrong_id_pre_order['pre_order'].fillna(0)

In [14]:
df_wrong_trans.to_csv("./df_wrong_trans.csv",index=False)
df_wrong_id_pre_order.to_csv("./df_wrong_id_pre_order.csv",index=False)

In [15]:
df_wrong_trans=pd.merge(df_wrong_trans,df_wrong_id_pre_order,on="customer_id_hashed",how="left")

df_delta=df_wrong_trans[['customer_id_hashed','trans_order_since_18Q1','pre_order']].drop_duplicates()
df_delta=df_delta.sort_values(['customer_id_hashed','trans_order_since_18Q1']).drop_duplicates("customer_id_hashed")
df_delta['delta']=df_delta['trans_order_since_18Q1']-df_delta['pre_order']
df_delta=df_delta[['customer_id_hashed','delta']]

In [16]:
df_wrong_trans=pd.merge(df_wrong_trans,df_delta,on="customer_id_hashed",how="left")
df_wrong_trans['fixed_order']=df_wrong_trans['trans_order_since_18Q1']-df_wrong_trans['delta']+1

In [17]:
df_wrong_trans.to_csv("./df_wrong_trans_corrected.csv",index=False)

In [21]:
df_wrong_trans[df_wrong_trans['customer_id_hashed']=="000027c1ec87c7ef710fe659c42190ce6dcf921e5309126a8cd20c401c55a600"]

,location_id,transaction_dt,transaction_id,customer_id_hashed,department_id,sales,units,trans_order_since_18Q1,pre_order,delta,fixed_order
12,1597,2020-06-11,201,000027c1ec87c7ef710fe659c42190ce6dcf921e530912...,364,30.0,1,12,10.0,2.0,11.0


In [23]:
df_wrong_trans['fixed_order']=df_wrong_trans['fixed_order'].astype(int)

del df_wrong_trans['trans_order_since_18Q1']
del df_wrong_trans['pre_order']
del df_wrong_trans['delta']
df_wrong_trans=df_wrong_trans.rename(columns={"fixed_order":"trans_order_since_18Q1"})

In [24]:
import gc
gc.collect()

2355

In [25]:
df_wrong_trans.to_csv("good_full_to_revise_write_in.csv",index=False)

In [27]:
a=pd.read_sql("desc Pred_POS_Department",con=BL_engine)
a

,Field,Type,Null,Key,Default,Extra
0,location_id,int(11),YES,MUL,None,
1,transaction_dt,date,YES,MUL,None,
2,transaction_id,varchar(16),YES,,None,
3,customer_id_hashed,char(64),YES,MUL,None,
4,department_id,varchar(16),YES,MUL,None,
5,sales,"decimal(10,2)",YES,,None,
6,units,int(11),YES,,None,
7,trans_order_since_18Q1,int(11),YES,MUL,None,


In [28]:
df_wrong_trans.columns.tolist()==a['Field'].values.tolist()

True

In [29]:
len(list_wrong_ids)

1404642

In [31]:
import os
writer_folder="./updated_sql_rows/"
os.mkdir(writer_folder)

In [37]:
for i in range(chunck_count):
    chunck_list=list_wrong_ids[i*chunck_size:(i+1)*chunck_size]
    str_list_wrong_ids=str(chunck_list).replace("[","(").replace("]",")")
    query_to_delete_in="delete from Pred_POS_Department where transaction_dt >= '2020-06-07' and customer_id_hashed in %s"%str_list_wrong_ids
    with BL_engine.connect() as connection:
        result = connection.execute(query_to_delete_in)
    
    df=df_wrong_trans[df_wrong_trans['customer_id_hashed'].isin(chunck_list)]
    df.to_sql("Pred_POS_Department",if_exists='append', con=BL_engine, index=False,chunksize=300000,
                    dtype={
                        'location_id':sqlalchemy.types.INTEGER(),
                        'transaction_dt':sqlalchemy.Date(), 
                        'transaction_id':sqlalchemy.types.VARCHAR(length=16),
                        'customer_id_hashed':sqlalchemy.types.VARCHAR(length=64),
                        'department_id':sqlalchemy.types.VARCHAR(length=16),
                        'sales':sqlalchemy.types.DECIMAL(precision=10,scale=2,asdecimal=True),
                        'units':sqlalchemy.types.INTEGER(),
                        'trans_order_since_18Q1':sqlalchemy.types.INTEGER()
                    })
    df.to_csv(writer_folder+"chunck_updated_%d.csv"%i,index=False)
    if i%10==1:
        print(i,datetime.datetime.now())

1 2020-08-12 01:06:15.549081
11 2020-08-12 01:32:09.496497
21 2020-08-12 01:46:31.316542
31 2020-08-12 01:59:56.116229
41 2020-08-12 02:12:35.024641
51 2020-08-12 02:22:04.639853
61 2020-08-12 02:31:15.072136
71 2020-08-12 02:40:28.192514
81 2020-08-12 02:49:46.082262
91 2020-08-12 02:59:08.023705
101 2020-08-12 03:08:22.475781
111 2020-08-12 03:18:11.411778
121 2020-08-12 03:27:42.578334
131 2020-08-12 03:36:56.257849
